## Deep Deterministic Policy Gradient

На этом семинаре мы будем обучать нейронную сеть на фреймворке __pytorch__ с помощью алгоритма Deep Deterministic Policy Gradient.

## Теория

Deep Deterministic Policy Gradient (DDPG) - это алгоритм, который одновременно учит Q-функцию и стратегию. Он использует off-policy данные и уравнения Беллмана для обучения Q-функции, а Q-функция используется для обучения стратегии.

Данный подход тесно связан с Q-обучением и мотивирован следующей идеей: если вы знаете оптимальную функцию action-value $Q^*(s,a)$, тогда для конкретного состояния, оптимальное действие $a^*(s)$ может быть найдено решением: 

$$a^*(s) = \arg \max_a Q^*(s,a).$$

Для сред с дискретным пространством действий - это легко, вычисляем полезности для каждого из действий, а потом берем максимум. Для непрерывных действий - это сложная оптимизационная задача.

DDPG чередует обучение аппроксиматора $Q^*(s,a)$ с обучением аппроксиматора  $a^*(s)$, и делает это специальным образом именно для непрерывных (continuous) сред, что отражается в том как алгоритм вычисляет $\max_a Q^*(s,a)$.
Поскольку пространство действий непрерывно, предполагается, что функция $Q^*(s,a)$ дифференцируема по аргументу действия. Это позволяет нам установить эффективное правило обучения на основе градиента для стратегии $\mu(s)$.

<img src="https://spinningup.openai.com/en/latest/_images/math/5811066e89799e65be299ec407846103fcf1f746.svg">

Оригинальная статья:  <a href="https://arxiv.org/abs/1509.02971">Continuous control with deep reinforcement learning Arxiv</a>

In [ ]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip install "gymnasium[classic-control, atari, accept-rom-license]" --quiet
    !pip install piglet --quiet
    !pip install imageio_ffmpeg --quiet
    !pip install moviepy==1.0.3 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 KB 34.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.5/67.5 KB 3.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import gymnasium as gym
import numpy as np
import random

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

### Вспомогательные функции

In [ ]:
def print_mean_reward(step, session_rewards, eval_session_rewards):
    if not session_rewards:
        return

    def get_mean_reward(rewards): 
        return round(sum(rewards) / len(rewards), 2)
    
    train_mean = get_mean_reward(session_rewards)
    eval_mean = get_mean_reward(eval_session_rewards)
    
    print(f"step: {str(step).zfill(6)}, train: {train_mean}, eval: {eval_mean}")
    return eval_mean


def to_tensor(x, dtype=np.float32):
    if isinstance(x, torch.Tensor):
        return x
    x = np.asarray(x, dtype=dtype)
    x = torch.from_numpy(x).to(device)
    return x

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Batch/replay buffer

Попробуйте сначала реализовать с обычным буфером для накопления прецедентов с момента последнего шага обновления агента. Затем там, где это возможно, дополните вашу реализацию более серьезной памятью прецедентов (опционально добавьте приоритизацию).
Задайте себе вопрос, можно ли использовать память прецедентов для обучения актора в DDPG? Является ли алгоритм DDPG off-policy алгоритмом и почему?

In [ ]:
####### Здесь ваш код ########
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    
    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)

        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done
    
    def __len__(self):
        return len(self.buffer)

    def reset(self):
        self.buffer.clear()
        self.position = 0

##############################

## Environment
### Нормализация пространства действий

In [ ]:
class NormalizedActions(gym.ActionWrapper):
    def action(self, action: np.ndarray) -> np.ndarray:
        low = self.action_space.low
        high = self.action_space.high

        scale_factor = (high - low) / 2
        reloc_factor = high - scale_factor

        action = action * scale_factor + reloc_factor
        action = np.clip(action, low, high)

        return action

    def reverse_action(self, action: np.ndarray) -> np.ndarray:
        low = self.action_space.low
        high = self.action_space.high

        scale_factor = (high - low) / 2
        reloc_factor = high - scale_factor

        action = (action - reloc_factor) / scale_factor
        action = np.clip(action, -1.0, 1.0)

        return action

### Исследование - GaussNoise
#### Добавляем Гауссовский шум к действиям детерминированной стратегии
Добавляем его только при обучении для исследования.

In [ ]:
class GaussNoise:
    def __init__(self, mu: float, sigma: float):
        self.mu = mu
        self.sigma = sigma
        self.dist = torch.distributions.Normal(self.mu, self.sigma)

    def sample(self, shape: torch.Size):
        noisy_action = self.dist.sample(shape).to(device)
        return noisy_action

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## DDPG Network 

<img src="https://raw.githubusercontent.com/Tviskaron/mipt/master/2020/RL/figures/DQN-DDPG.svg">

### DDPG Model
Реализуйте модель актор-критика `DdpgModel`. Скорее всего вам понадобится `nn.Module`. Можете реализовать актор-критика единым модулем, а можете разнести их (первый вариант не обязательно предполагает общее тело сетей).

In [ ]:
class DDPGModel(nn.Module):
    def __init__(
            self,
            num_inputs,
            num_actions,
            hidden_size,
            noise_params: dict
    ):
        super().__init__()
        self.value_net = nn.Sequential(
            nn.Linear(num_inputs + num_actions, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1)
        )
        
        self.policy_net = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, num_actions),
            nn.Tanh()
        )
        self.noise_model = GaussNoise(**noise_params)

    def forward(self, state, action):
        return (self.value_forward(state, action), 
               self.policy_forward(state).detach().cpu().numpy()[0])

    def value_forward(self, state, action):
        state = to_tensor(state); action = to_tensor(action)

        if len(state.shape) == 1:
            state=state.unsqueeze(0)

        if len(action.shape) == 1:
            action=action.unsqueeze(0)
        
        x = torch.cat([state, action], -1)
        value = self.value_net(x)
        return value

    def policy_forward(self, state):
        
         state = to_tensor(state)
         if len(state.shape) == 1:
            state=state.unsqueeze(0)
         action = self.policy_net(state)
         
         if self.training:
            noise = self.noise_model.sample(action.size())
            selected_action = torch.clip(action + noise, min=-1.0, max=1.0) 

         return action

    def soft_copy(self, other, v_tau: float, p_tau: float):
        for target_param, param in zip(other.policy_net.parameters(), self.policy_net.parameters()):
            target_param.data.copy_(
                target_param.data * (1.0 - p_tau) + param.data * p_tau
            )
        for target_param, param in zip(other.value_net.parameters(), self.value_net.parameters()):
            target_param.data.copy_(
                target_param.data * (1.0 - v_tau) + param.data * v_tau
            )
        return self

##############################

### DDPG Agent

Реализуйте класс агента `DdpgAgent`, который содержит:

- обучаемую модель актор-критика и его оптимизаторы (например, `torch.optim.Adam`)
- периодически обновляемую копию модели с замороженными весами для вычисления TD target.
- метод `act` для выбора действия. Можете добавить флаг `learn` для подмешивания гауссовского шума при обучении, либо добавляйте его снаружи агента.
- метод `learn` для обновления модели агента (и актора, и критика) на новом пакете опыта.

In [ ]:
from copy import deepcopy
class DDPGAgent:
    def __init__(self, 
                 state_dim: int, 
                 hidden_dim: int, 
                 action_dim: int, 
                 lr: dict, 
                 gamma: float, 
                 taus: dict,
                 noise_params: dict):
        self.lr = lr
        self.gamma = gamma
        self.tau = taus

        self.agent = DDPGModel(state_dim, action_dim, hidden_dim, noise_params).to(device)
        self.target_net = DDPGModel(state_dim, action_dim, hidden_dim, noise_params).to(device).soft_copy(self.agent, 1., 1.)
        self.value_optimizer = torch.optim.Adam([
            {"params": self.agent.value_net.parameters(), "lr": self.lr["valuef"]}])
        
        self.policy_optimizer =  torch.optim.Adam([ 
            {"params": self.agent.policy_net.parameters(), "lr": self.lr["policy"]}])
        
        self.min_max_value = {"min": -np.inf, "max": np.inf}    
        self.action_dim = action_dim
        
    def act(self, state, action):
        ####### Здесь ваш код ########
        value, action = self.agent(state, action)
        return value, action
        ##############################

    def get_action(self, state):
        return self.agent.policy_forward(state).item()

    def update(self, batched_rollout: tuple):
        state, action, reward, next_state, terminal = batched_rollout
        action = to_tensor(action).reshape(-1, 1)
        reward = to_tensor(reward).reshape(-1, 1)
        terminal = to_tensor(terminal, dtype=np.int8).reshape(-1, 1)


        policy_loss = self.agent.value_forward(state, self.agent.policy_forward(state))
        policy_loss = -policy_loss.mean()
        
        next_action = self.target_net.policy_forward(next_state)
        target_next_value = self.target_net.value_forward(next_state, next_action.detach())
        ####### Здесь ваш код ########
        target = reward + self.gamma * (1. - terminal) * target_next_value
        ##############################
        
        ####### Здесь ваш код ########
        current_value = self.agent.value_forward(state, action)
        ##############################
    
        ####### Здесь ваш код ########
        value_loss = nn.MSELoss(reduction = "mean")(current_value, target.detach())
        ##############################
        
        self.policy_optimizer.zero_grad()
        policy_loss.backward()
        self.policy_optimizer.step()

        self.value_optimizer.zero_grad()
        value_loss.backward()
        self.value_optimizer.step()
        
        self.agent = self.agent.soft_copy(self.target_net, self.tau["valuef"], self.tau["policy"])
##############################

## Evaluation

Реализуйте функцию `run`, которая принимает среду, гиперпараметры агента и условие останова эксперимента (return threshold $G_{target}$). Используйте функцию `print_mean_reward` для вывода промежуточных результатов качества агента в трейн и eval режимах.

Проведите эксперимент на среде с непрерывным пространством действий (например, continuous montain car или pendulum).

In [ ]:
from dataclasses import dataclass
@dataclass
class ExperimentReplay:
    agent: DDPGAgent
    replay_buffer: ReplayBuffer
    batch_size: int
    train_schedule: int

    def generate_session(self, T: int, train: bool = False) -> int:
        total_reward = 0
        state, _ = env.reset()
        self.agent.agent.train(train)
        for t in range(T):
            action = self.agent.get_action(state)
            next_state, reward, terminated, info = env.step(action)
            # terminated |= truncated
            if train:
                self.replay_buffer.push(state, action, reward, next_state, terminated)
                if len(self.replay_buffer) > self.batch_size and t % self.train_schedule == 0:
                    states, actions, rewards, next_states, terminals = self.replay_buffer.sample(self.batch_size)
                    self.agent.update((states, actions, rewards, next_states, terminals))

            total_reward += reward
            state = next_state
            if terminated:
                break

        return total_reward

In [ ]:
import gymnasium as gym
from gym.wrappers import TimeLimit
# env_name, success_reward = "MountainCarContinuous", 100
env_name, success_reward = "Pendulum-v0", -100
env = NormalizedActions(TimeLimit(gym.make("MountainCarContinuous-v0"), 1000))

noise_params = {"mu": 0.01, "sigma": 0.3}
lrs = {"valuef": 1e-3, "policy": 1e-4}
taus = {"valuef": .8, "policy": .85}
batch_size = 100
gamma = 0.99
state_dim  = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
hidden_dim = 256
max_steps = 500

agent = DDPGAgent(state_dim=state_dim, 
                 hidden_dim=hidden_dim, 
                 action_dim=action_dim, 
                 lr = lrs, 
                 gamma = 0.99, 
                 taus = taus,
                 noise_params = noise_params)

memory = ReplayBuffer(1000)
exp= ExperimentReplay(agent = agent, replay_buffer=memory, batch_size =batch_size, train_schedule = 16)

valid_mean_rewards = []
for i in range(100):    
    session_rewards_train = [
        exp.generate_session(max_steps, train=True) 
        for _ in range(10)
    ]
    session_rewards_valid = [
        exp.generate_session(max_steps, train=False) 
        for _ in range(10)
    ]
    print(
        "epoch #{:02d}\tmean reward (train) = {:.3f}\tmean reward (valid) = {:.3f}".format(
        i, np.mean(session_rewards_train), np.mean(session_rewards_valid))
    )

    valid_mean_rewards.append(np.mean(session_rewards_valid))
    if len(valid_mean_rewards) > 5 and np.mean(valid_mean_rewards[-5:]) > success_reward:
        print("Pendulum решен!")
        break

env.close()



epoch #00	mean reward (train) = -40.327	mean reward (valid) = -48.810
epoch #01	mean reward (train) = -48.412	mean reward (valid) = -38.473
epoch #02	mean reward (train) = -9.119	mean reward (valid) = -0.912
epoch #03	mean reward (train) = -0.626	mean reward (valid) = -0.203
epoch #04	mean reward (train) = -0.095	mean reward (valid) = -0.088
epoch #05	mean reward (train) = -0.091	mean reward (valid) = -0.102
Pendulum решен!
